In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../scripts'))
if module_path not in sys.path:
    sys.path.append(module_path)
import query_database as function
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder

In [2]:
ratings = pd.read_csv('data/ml-latest/ratings.csv')
movies = pd.read_csv('data/ml-latest/movies.csv')
links = pd.read_csv('data/ml-latest/links.csv')
tags = pd.read_csv('data/ml-latest/tags.csv')

In [3]:
movies[movies['title'].str.contains('Lord of the Rings')]

,movieId,title,genres
2033,2116,"Lord of the Rings, The (1978)",Adventure|Animation|Children|Fantasy
4898,4993,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy
5854,5952,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy
7042,7153,"Lord of the Rings: The Return of the King, The...",Action|Adventure|Drama|Fantasy


In [4]:
links[links['movieId'] == 7153]

,movieId,imdbId,tmdbId
7042,7153,167260,122.0


In [114]:
links[links['tmdbId'] == 10480]

,movieId,imdbId,tmdbId
3801,3893,171580,10480.0


In [6]:
tags.head()

,userId,movieId,tag,timestamp
0,14,110,epic,1443148538
1,14,110,Medieval,1443148532
2,14,260,sci-fi,1442169410
3,14,260,space action,1442169421
4,14,318,imdb top 250,1442615195


In [7]:
drop_cols = ['tagline', 'imdb_avgRating', 'imdb_numVotes', 'poster_path', 'num_of_cast', 'top_10_cast_popularity_mean'
             'homepage', 'budget', 'revenue', 'imdb_numVotes', 'top_10_cast_popularity', 'top_10_crew_popularity_mean'
             'top_10_crew_popularity', 'originalTitle', 'belongs_to_collection', 'release_date']
combined_db = function.get_combined(release_year_cutoff=2000, drop_lst=drop_cols)

In [8]:
combined_db = combined_db.drop(columns = ['homepage', 'top_10_cast_popularity_mean', 
                                          'num_of_crew', 'top_10_crew_popularity', 
                                          'status', 'top_10_crew_popularity_mean'])

In [9]:
combined_db.shape

(110889, 19)

In [10]:
# preprocessing
combined = combined_db.copy()
combined['runtimeMinutes'] = combined['runtimeMinutes'].fillna(combined["runtimeMinutes"].median())
combined=combined.replace({'runtimeMinutes': {0: combined["runtimeMinutes"].median()}}) 
# combined['genres'] = combined['genres'].fillna('None')
combined['Writers'] = combined['Writers'].fillna('None')
combined['Directors'] = combined['Directors'].fillna('None')
combined['casts'] = combined['casts'].fillna('None')
# combined['keywords_name'] = combined['keywords_name'].fillna('None')
combined['company_name'] = combined['company_name'].fillna('None')
combined['overview'] = combined['overview'].fillna('')
combined['casts'] = combined['casts'].apply(lambda x: x[:3] if len(x) >=3 else x)

# drop rows with the same title
combined = combined.drop_duplicates(subset='primaryTitle', keep="first")
combined = combined.drop(columns = ['keywords_name', 'genres'])
combined = combined.reset_index()

In [52]:
def make_genresList(x):
    gen = []
    st = " "
    if not isinstance(x, list):
        if x == 'None':
            return ''
        else:
            gen.append(x)
            return (st.join(gen))
    for i in x:
        if i == 'Science Fiction':
            scifi = 'Sci-Fi'
            gen.append(scifi)
        else:
            gen.append(i)
    if gen == []:
        return ''
    else:
        return (st.join(gen))

In [11]:
def process_list(x):
    gen = []
    st = " "
    if not isinstance(x, list):
        if x == 'None':
            return ''
        else:
            gen.append(x)
            return (st.join(gen))
    for i in x:
        gen.append(i)
    if gen == [] or gen == '[]':
        return ''
    else:
        return (st.join(gen))

In [12]:
# combined['genres'] = combined['genres'].map(lambda x: make_genresList(x))
combined['Directors'] = combined['Directors'].map(lambda x: process_list(x))
combined['casts'] = combined['casts'].map(lambda x: process_list(x))
combined['Writers'] = combined['Writers'].map(lambda x: process_list(x))
combined['company_name'] = combined['company_name'].map(lambda x: process_list(x))
# combined['keywords_name'] = combined['keywords_name'].map(lambda x: process_list(x))
# don't need crews


In [13]:
combined['casts'].values[1000]

'Ashley Williams Omar Benson Miller E-40'

In [14]:
combined['document'] = combined[['Directors', 'casts', 'Writers',
                                 'company_name', 'overview']].apply(lambda x: ' '.join(x), axis=1)
combined = combined.astype({"isAdult": int})

In [15]:
combined.head()

,index,_id,primaryTitle,isAdult,release_year,runtimeMinutes,Directors,Writers,casts,tmdb_id,original_language,popularity,overview,company_name,crews,tmdb_avgRating,tmdb_numVotes,release_year_int,document
0,0,tt0011216,Spanish Fiesta,0,2019.0,67.0,Germaine Dulac,Louis Delluc,Ève Francis Gabriel Gabrio Gaston Modot,364671,fr,0.6,A Spanish festival reveals the emotional dista...,Les Films Louis Nalpas,"[Germaine Dulac, Serge Sandberg, Paul Parguel,...",6.0,1,2019,Germaine Dulac Ève Francis Gabriel Gabrio Gast...
1,1,tt0011801,Tötet nicht mehr,0,2019.0,127.0,,Gerhard Lamprecht,Emilie Kurz Rudolf Klein-Rhoden Paul Rehkopf,611205,de,0.6,The director and co-writer Lupu Pick plays mus...,Rex-Film GmbH,"[Gerhard Lamprecht, Willi Herrmann, Lupu Pick,...",0.0,0,2019,Emilie Kurz Rudolf Klein-Rhoden Paul Rehkopf ...
2,2,tt0015414,La tierra de los toros,0,2000.0,60.0,Musidora,,Musidora Antonio Cañero,607290,es,0.6,,Musidora Films,[Musidora],0.0,0,2000,Musidora Musidora Antonio Cañero Musidora Films
3,3,tt0016906,Frivolinas,0,2014.0,80.0,Arturo Carballo,,María Caballé José López Alonso Ramón Álvarez ...,400531,es,0.6,Don Casto is a widower who spends most of his ...,Ediciones Seleccionadas Arturo Carballo,"[Arturo Carballo, Ramón de Baños]",0.0,0,2014,Arturo Carballo María Caballé José López Alons...
4,4,tt0019996,Hongxia,0,2011.0,94.0,Wen Yi-Min,,Xuepeng Fan Chu Shao-Chuen Wen Yi-Min,267384,zh,0.6,Red Heroine was a smash hit on release and a p...,Youlian Film Company,"[Shiquan Yao, Wen Yi-Min, Shang Guan-Wu]",6.3,3,2011,Wen Yi-Min Xuepeng Fan Chu Shao-Chuen Wen Yi-M...


In [46]:
new_df = combined.copy()
new_df = new_df.drop(columns = ['release_year', 'casts', 'original_language',
                                'overview', 'runtimeMinutes', 'tmdb_numVotes',
                                'crews', 'index', 'isAdult'])
new_df = new_df.reset_index()
new_df.shape

(103215, 11)

In [47]:
new_df.head()

,index,_id,primaryTitle,Directors,Writers,tmdb_id,popularity,company_name,tmdb_avgRating,release_year_int,document
0,0,tt0011216,Spanish Fiesta,Germaine Dulac,Louis Delluc,364671,0.6,Les Films Louis Nalpas,6.0,2019,Germaine Dulac Ève Francis Gabriel Gabrio Gast...
1,1,tt0011801,Tötet nicht mehr,,Gerhard Lamprecht,611205,0.6,Rex-Film GmbH,0.0,2019,Emilie Kurz Rudolf Klein-Rhoden Paul Rehkopf ...
2,2,tt0015414,La tierra de los toros,Musidora,,607290,0.6,Musidora Films,0.0,2000,Musidora Musidora Antonio Cañero Musidora Films
3,3,tt0016906,Frivolinas,Arturo Carballo,,400531,0.6,Ediciones Seleccionadas Arturo Carballo,0.0,2014,Arturo Carballo María Caballé José López Alons...
4,4,tt0019996,Hongxia,Wen Yi-Min,,267384,0.6,Youlian Film Company,6.3,2011,Wen Yi-Min Xuepeng Fan Chu Shao-Chuen Wen Yi-M...


In [48]:
new_df.tmdb_id.nunique()

103215

In [51]:
tmdb_ids = new_df.tmdb_id.unique()
links_new = links[links['tmdbId'].isin(tmdb_ids)]
links_new.tmdbId.nunique()

22275

In [57]:
links_new = links_new.drop_duplicates(subset='tmdbId', keep="first")

In [58]:
links_new.movieId.nunique()

22275

In [59]:
links_new.head()

,movieId,imdbId,tmdbId
711,723,92123,131232.0
878,895,105729,79782.0
2594,2679,124555,84198.0
2684,2769,138946,19457.0
3091,3177,195945,10471.0


In [60]:
links_new[links_new['movieId'] == 7153]

,movieId,imdbId,tmdbId
7042,7153,167260,122.0


In [61]:
movieIds = links_new.movieId.unique()
ratings_new = ratings[ratings['movieId'].isin(movieIds)]
ratings_new.movieId.nunique()

21639

In [62]:
ratings_new.head()

,userId,movieId,rating,timestamp
14,1,3826,2.0,1256677210
15,1,3893,3.5,1256677486
453,4,3190,3.5,1113766176
468,4,3298,4.5,1113766820
469,4,3300,3.5,1113766824


In [63]:
movieIds = ratings_new.movieId.unique()
movies_new = movies[movies['movieId'].isin(movieIds)]
movies_new.movieId.nunique()

21639

In [64]:
movies_new.head()

,movieId,title,genres
711,723,Two Friends (1986),Drama
878,895,Venice/Venice (1992),Drama
2594,2679,Finding North (1998),Comedy|Drama|Romance
2684,2769,"Yards, The (2000)",Crime|Drama
3091,3177,Next Friday (2000),Comedy


In [65]:
movieIds = ratings_new.movieId.unique()
tags_new = tags[tags['movieId'].isin(movieIds)]
tags_new.movieId.nunique()

16421

In [66]:
tags_new.head()

,userId,movieId,tag,timestamp
10,14,7458,Epic,1443148675
12,14,117529,dinosaurs,1443148646
17,42,37733,disappointing,1264106059
18,42,37733,overrated,1264106052
19,42,37733,stupid,1264106067


In [108]:
tags_new.to_csv('data/tags_new.csv', index=False)

In [67]:
new_df = new_df.merge(links_new, left_on='tmdb_id', right_on='tmdbId')

In [68]:
new_df.drop(['imdbId', '_id', 'tmdbId'], axis=1, inplace=True)

In [69]:
movieIds = ratings_new.movieId.unique()
new_df = new_df[new_df['movieId'].isin(movieIds)]
new_df.movieId.nunique()

21639

In [70]:
new_df.tmdb_id.nunique()

21639

In [71]:
all_movies_tmdbId = new_df.tmdb_id.unique()
with open('./pkl/all_movies_tmdbId.pkl', 'wb') as fh:
    pickle.dump(all_movies_tmdbId, fh)

In [81]:
new_df.head()

,index,primaryTitle,Directors,Writers,tmdb_id,popularity,company_name,tmdb_avgRating,release_year_int,document,movieId
0,5,Kate & Leopold,James Mangold None,Steven Rogers,11232,12.086,Konrad Pictures Miramax,6.3,2001,James Mangold None Hugh Jackman Meg Ryan Liev ...,4992
1,10,The Other Side of the Wind,Orson Welles None,Dax Phelan,299782,11.076,SACI Les Films de l'Astrophore,6.8,2018,Orson Welles None Orson Welles John Huston Den...,159163
2,14,Crime and Punishment,Menahem Golan None,Fyodor Dostoevsky Ivan Mendzheritskiy,109809,7.465,,5.5,2002,Menahem Golan None John Hurt Crispin Glover Va...,116293
3,17,Master i Margarita,Yuriy Kara,Mikhail A. Bulgakov Felix Kroll,63163,2.223,Tvorcheskaya Assotsiatsiya Mezhdunarodnykh Pro...,6.2,2006,Yuriy Kara Mikhail Ulyanov Anastasiya Vertinsk...,167420
4,18,The Fantasticks,Michael Ritchie,Harvey Schmidt Tom Jones,62127,5.170,Sullivan Street Productions Michael Ritchie Pr...,5.5,2000,Michael Ritchie Brad Sullivan Jean Louisa Kell...,4193


In [94]:
combined_genres = new_df.merge(movies_new, on='movieId')

In [96]:
combined_genres['genres'] = combined_genres['genres'].str.replace(pat="|", repl=" ", regex=False)

In [100]:
combined_genres.head()

,index,primaryTitle,Directors,Writers,tmdb_id,popularity,company_name,tmdb_avgRating,release_year_int,document,movieId,title,genres
0,5,Kate & Leopold,James Mangold None,Steven Rogers,11232,12.086,Konrad Pictures Miramax,6.3,2001,James Mangold None Hugh Jackman Meg Ryan Liev ...,4992,Kate & Leopold (2001),Comedy Romance
1,10,The Other Side of the Wind,Orson Welles None,Dax Phelan,299782,11.076,SACI Les Films de l'Astrophore,6.8,2018,Orson Welles None Orson Welles John Huston Den...,159163,The Other Side of the Wind (2016),Comedy Drama
2,14,Crime and Punishment,Menahem Golan None,Fyodor Dostoevsky Ivan Mendzheritskiy,109809,7.465,,5.5,2002,Menahem Golan None John Hurt Crispin Glover Va...,116293,Crime and Punishment (2002),Drama
3,17,Master i Margarita,Yuriy Kara,Mikhail A. Bulgakov Felix Kroll,63163,2.223,Tvorcheskaya Assotsiatsiya Mezhdunarodnykh Pro...,6.2,2006,Yuriy Kara Mikhail Ulyanov Anastasiya Vertinsk...,167420,The Master and Margarita (1994),Drama Fantasy Mystery Romance
4,18,The Fantasticks,Michael Ritchie,Harvey Schmidt Tom Jones,62127,5.170,Sullivan Street Productions Michael Ritchie Pr...,5.5,2000,Michael Ritchie Brad Sullivan Jean Louisa Kell...,4193,"Fantasticks, The (1995)",Musical


In [101]:
combined_genres['document'] = combined_genres[['document', 'genres']].apply(lambda x: ' '.join(x), axis=1)

In [107]:
combined_genres.to_csv('data/combined_genres.csv', index=False)

In [73]:
print("{} unique movies in dataset".format(len(new_df.primaryTitle.unique())))

21639 unique movies in dataset


In [74]:
movie_unique_ids = ratings_new.movieId.unique()
movie_to_index = {o:i for i,o in enumerate(movie_unique_ids)}
# ratings_new['movie'] = ratings_new['movieId'].apply(lambda x: movie_to_index[x])

In [75]:
user_unique_ids = ratings_new.userId.unique()
user_to_index = {o:i for i,o in enumerate(user_unique_ids)}
# ratings_new['user'] = ratings_new['userId'].apply(lambda x: user_to_index[x])

In [77]:
with open('./pkl/movie_to_index.pkl', 'wb') as fh:
    pickle.dump(movie_to_index, fh)

In [78]:
with open('./pkl/user_to_index.pkl', 'wb') as fh:
    pickle.dump(user_to_index, fh)

In [79]:
ratings_new.head()

,userId,movieId,rating,timestamp
14,1,3826,2.0,1256677210
15,1,3893,3.5,1256677486
453,4,3190,3.5,1113766176
468,4,3298,4.5,1113766820
469,4,3300,3.5,1113766824


In [80]:
ratings_new.to_csv('data/ratings_new.csv', index=False)
movies_new.to_csv('data/movies_new.csv', index=False)

In [112]:
movie_to_index

{3826: 0,
 3893: 1,
 3190: 2,
 3298: 3,
 3300: 4,
 3301: 5,
 3316: 6,
 3354: 7,
 3452: 8,
 3481: 9,
 3513: 10,
 3555: 11,
 3578: 12,
 3593: 13,
 3617: 14,
 3623: 15,
 3717: 16,
 3745: 17,
 3751: 18,
 3753: 19,
 3755: 20,
 3793: 21,
 3825: 22,
 3827: 23,
 3879: 24,
 3889: 25,
 3916: 26,
 3948: 27,
 3952: 28,
 3977: 29,
 3980: 30,
 3981: 31,
 3990: 32,
 3994: 33,
 3996: 34,
 3997: 35,
 4011: 36,
 4015: 37,
 4018: 38,
 4019: 39,
 4022: 40,
 4025: 41,
 4034: 42,
 4052: 43,
 4148: 44,
 4167: 45,
 4223: 46,
 4226: 47,
 4232: 48,
 4238: 49,
 4239: 50,
 4246: 51,
 4270: 52,
 4306: 53,
 4308: 54,
 4343: 55,
 4344: 56,
 4367: 57,
 4446: 58,
 4448: 59,
 4638: 60,
 4643: 61,
 4701: 62,
 4718: 63,
 4719: 64,
 4721: 65,
 4734: 66,
 4776: 67,
 4844: 68,
 4874: 69,
 4878: 70,
 4886: 71,
 4889: 72,
 4890: 73,
 4896: 74,
 4901: 75,
 4958: 76,
 4963: 77,
 4975: 78,
 4979: 79,
 4992: 80,
 4993: 81,
 4995: 82,
 5010: 83,
 5064: 84,
 5107: 85,
 5110: 86,
 5152: 87,
 5218: 88,
 5219: 89,
 5254: 90,
 5283: 91